1. Install comyx
2. Define Simulation Parameters (num_users, channel_model, ...)
3. Generate User Profiles
4. Simulate NOMA Transmissions
5. Format the results
6. Save the dataset

In [1]:
!pip install comyx
!pip install tqdm

#Importing Necessary Libraries

In [2]:
from comyx.network import UserEquipment, BaseStation, Link
from comyx.propagation import get_noise_power
from comyx.utils import dbm2pow, get_distance, generate_seed, db2pow
from scipy.optimize import minimize


import numpy as np
from numba import jit
from matplotlib import pyplot as plt

#Setting Up Environment

In [3]:
Pt = np.linspace(-10, 30, 80)  # dBm
#Pt = np.array([20]) # dBm
Pt_lin = dbm2pow(Pt)  # Watt
bandwidth = 1e7  # Bandwidth in Hz
frequency = 2.4e9  # Carrier frequency
temperature = 300  # Kelvin # 300
mc = 1  # Number of channel realizations
simulation_area_size = 60  # Size of square area (units)
max_distance_bs_ue = 30     # Maximum distance from BS to each UE (units) # 50
max_distance_ue_ue = 20      # Maximum distance between UEs (units) # 20
min_distance_ue_ue = 10       # Minimum distance between UEs # 15

allocation_factors = np.linspace(0.01, 0.3, 200) # All allocation factors for UEn and UEf

N0 = get_noise_power(temperature, bandwidth)  # dBm
N0_lin = dbm2pow(N0)  # Watt

R_prime_n = 2.5
R_prime_f = 2.5

n_antennas = 2

fading_args = {"type": "rayleigh", "sigma": 1 / 2}
pathloss_args = {
    "type": "reference",
    "alpha": 3.5, #3.5
    "p0": 40, # 20
    "frequency": frequency,
}  # p0 is the reference power in dBm

# Network Setup

In [4]:
# Function to check feasibility of generated locations for UEs
def is_feasible(bs_position, ue1_position, ue2_position, size_area, max_dist_bs_ue, max_dist_ue_ue, min_dist_ue_ue):
    # Check if BS and UEs are within the area bounds
    if (abs(bs_position[0]) > size_area / 2 or abs(bs_position[1]) > size_area / 2 or
        abs(ue1_position[0]) > size_area / 2 or abs(ue1_position[1]) > size_area / 2 or
        abs(ue2_position[0]) > size_area / 2 or abs(ue2_position[1]) > size_area / 2):
        return False

    # Check distances from BS to UEs
    if (get_distance(bs_position, ue1_position) > max_dist_bs_ue or
        get_distance(bs_position, ue2_position) > max_dist_bs_ue):
        return False

    # Check distance between UEs
    if get_distance(ue1_position, ue2_position) > max_dist_ue_ue or get_distance(ue1_position, ue2_position) < min_dist_ue_ue:
        return False

    # ensure UEn is closer to base station that UEf
    if get_distance(ue1_position, bs_position) > get_distance(ue2_position, bs_position):
        # print(False)
        return False

    return True


#Initialize Links

In [5]:
def initialize_links(BS, UEn, UEf):
  # Shapes for channels
  shape_bu = (n_antennas, n_antennas, mc)

  # Links
  # fmt: off
  link_bs_uen = Link(
      BS, UEn,
      fading_args, pathloss_args,
      shape=shape_bu, seed=generate_seed("BS-UEn"),
  )

  link_bs_uef = Link(
      BS, UEf,
      fading_args, pathloss_args,
      shape=shape_bu, seed=generate_seed("BS-UEf"),
  )

  return link_bs_uen, link_bs_uef

#Initialize Network

In [19]:
# Initialize Setup
def initialize():
  # Initialize Base Station position
  BS = BaseStation("BS", position=[0, 0, 10], n_antennas=n_antennas, t_power=Pt_lin)

  # Loop until valid positions for UEn and UEf are found
  valid_positions_found = False
  channel_difference = False
  while not valid_positions_found and not channel_difference:
      # Generate random positions for UEn and UEf within specified constraints
      UEn_position = [np.random.uniform(-max_distance_bs_ue, max_distance_bs_ue),
                      np.random.uniform(-max_distance_bs_ue, max_distance_bs_ue),
                      1]

      UEf_position = [np.random.uniform(-max_distance_bs_ue, max_distance_bs_ue),
                      np.random.uniform(-max_distance_bs_ue, max_distance_bs_ue),
                      1]


      # Check if all positions are feasible
      valid_positions_found = is_feasible(BS.position, UEn_position, UEf_position,
                                          simulation_area_size, max_distance_bs_ue, max_distance_ue_ue, min_distance_ue_ue)

      # Initialize User Equipments with their feasible positions
      UEn = UserEquipment("UEn", position=UEn_position, n_antennas=n_antennas)
      UEf = UserEquipment("UEf", position=UEf_position, n_antennas=n_antennas)

      # Initialize Links
      link_bs_uen, link_bs_uef = initialize_links(BS, UEn, UEf)

      # Get channel gains
      gain_f = link_bs_uef.magnitude**2
      gain_n = link_bs_uen.magnitude**2

      # channel difference
      # channel_difference =  np.all(gain_n > gain_f)
      channel_difference = np.mean(gain_n) > np.mean(gain_f)


  return BS, UEn, UEf, link_bs_uen, link_bs_uef, gain_f, gain_n

#Compute Rates & Spectral Efficiency

In [20]:
import numpy as np

def computeRates(BS, UEn, UEf, allocation_factors, gain_n, gain_f):
    results = []
    sample_index = 0

    # Looping over each allocation factor
    for alloc_UEn in allocation_factors:
        alloc_UEf = 1 - alloc_UEn

        # Update the power allocations for BS
        BS.allocations = {"UEn": alloc_UEn, "UEf": alloc_UEf}

        UEn.sinr_pre = np.zeros((len(Pt), mc))
        UEn.sinr = np.zeros((len(Pt), mc))
        UEf.sinr = np.zeros((len(Pt), mc))
        SE_nf = np.zeros((len(Pt), mc))
        SE_n = np.zeros((len(Pt), mc))
        SE_f = np.zeros((len(Pt), mc))
        SE_total = np.zeros((len(Pt), mc))
        SE_difference = np.zeros((len(Pt), mc))
        R_n = np.zeros((len(Pt), mc))
        R_f = np.zeros((len(Pt), mc))

        # Loop over each power level
        for i, p in enumerate(Pt_lin):
            p = BS.t_power[i]

            # Loop over each channel realization
            for k in range(mc):
                sample_index += 1

                gain_f_scalar = gain_f[0, 0, k]
                gain_n_scalar = gain_n[0, 0, k]

                # Effective transmit powers for each user
                P_n = BS.allocations["UEn"] * p  # Effective power for near user
                P_f = BS.allocations["UEf"] * p  # Effective power for far user

                # SINR calculations
                SINR_n = (P_n * gain_n_scalar) / N0_lin  # Near user SINR (after SIC)
                SINR_f = (P_f * gain_f_scalar) / (P_n * gain_f_scalar + N0_lin)  # Far user SINR

                # Data rates (using Shannon capacity formula)
                R_n[i, k] = np.log2(1 + SINR_n)  # Data rate for near user
                R_f[i, k] = np.log2(1 + SINR_f)  # Data rate for far user

                # QoS check
                if R_n[i, k] < R_prime_n or R_f[i, k] < R_prime_f:
                    # Skip this iteration if QoS is not met
                    SE_n[i, k] = np.nan  # Mark as invalid
                    SE_f[i, k] = np.nan  # Mark as invalid
                    SE_total[i, k] = np.nan  # Mark as invalid
                    SE_difference[i, k] = np.nan  # Mark as invalid
                    continue

                # Spectral efficiencies
                SE_n[i, k] = np.log2(1 + SINR_n)  # Near user SE
                SE_f[i, k] = np.log2(1 + SINR_f)  # Far user SE

                # Total spectral efficiency
                SE_total[i, k] = SE_n[i, k] + SE_f[i, k]

                # Difference in SE
                SE_difference[i, k] = abs(SE_n[i, k] - SE_f[i, k])

        # Find the index where the SE difference is minimized (ignoring NaN values)
        valid_SE_difference = np.where(np.isnan(SE_difference), np.inf, SE_difference)  # Replace NaN with infinity
        min_diff_index = np.unravel_index(np.argmin(valid_SE_difference), SE_difference.shape)

        # Check if a valid result was found
        if np.isinf(valid_SE_difference[min_diff_index]):
            # No valid results for this allocation factor
            continue

        # Extract the minimum SE difference and corresponding values
        min_SE_diff = SE_difference[min_diff_index]
        optimal_power_index = min_diff_index[0]
        optimal_channel_index = min_diff_index[1]
        optimal_power = Pt[optimal_power_index]
        optimal_SE_n = SE_n[min_diff_index]
        optimal_SE_f = SE_f[min_diff_index]
        optimal_SE_total = SE_total[min_diff_index]
        optimal_gain_f = gain_f[0, 0, optimal_channel_index]
        optimal_gain_n = gain_n[0, 0, optimal_channel_index]

        # Store results including allocation coefficients and minimized SE difference
        results.append({
            'sample index': sample_index,
            'alloc_UEn': alloc_UEn,
            'alloc_UEf': alloc_UEf,
            'operational_power': int(optimal_power),
            "SE_n": optimal_SE_n,
            "SE_f": optimal_SE_f,
            'SE_difference': min_SE_diff,
            'max_spectral_efficiency': optimal_SE_total,
            'optimal_gain_f': optimal_gain_f,
            'optimal_gain_n': optimal_gain_n,
            'optimal_channel_index': optimal_channel_index,
            'optimal_power_index': optimal_power_index,
        })

    return results

# Optimal Channel Realization

In [8]:
def find_optimal_results(results):
  # Initialize an empty list to store the optimal results
  optimal_results = []

  # Find the entry with the maximum energy efficiency
  # best_result = max(results, key=lambda x: x['max_spectral_efficiency'])
  best_result = min(results, key=lambda x: x['SE_difference'])

  # Append the best result to the optimal_results array
  optimal_results.append(best_result)

  return optimal_results


In [9]:
# Print the optimal results
def print_optimal_results(optimal_results):
  for res in optimal_results:
      print(f"Optimal Allocation Factors: UEn={res['alloc_UEn']:.5f}, UEf={res['alloc_UEf']:.2f}")
      print("Operational Power:", res['operational_power'])
      print("Maximum Spectral Efficiency:", res['max_spectral_efficiency'])
      print("Optimal Channel Realization Values:")
      print("  Gain for UEf (edge user):", res['optimal_gain_f'])
      print("  Gain for UEn (center user):", res['optimal_gain_n'])

In [10]:
# Print ALL the results
def printResults(results):
  for idx, res in enumerate(results):
      print(f"Result {idx + 1}:")
      print(f"\tAllocation Factors:")
      print(f"\t  UEn: {res['alloc_UEn']:}")
      print(f"\t  UEf: {res['alloc_UEf']:}")
      print(f"\tOperational Power: {res['operational_power']} dBm")
      print(f"\tMaximum Spectral Efficiency: {res['max_spectral_efficiency']:} bps/Hz")
      print(f"\tOptimal Channel Realization Values:")
      print(f"\t  Gain for UEf (edge user): {res['optimal_gain_f']:}")
      print(f"\t  Gain for UEn (center user): {res['optimal_gain_n']:}")
      print("\n" + "-" * 40 + "\n")

# Running Code


In [ ]:
import json
from tqdm import tqdm

# Function to write JSON data to a single file
def write_json_file(data, filename):
    with open(filename, 'w') as json_file:
        json.dump(data, json_file, indent=4)

# Initialize a list to store all runs' data
data = []

# Number of rows of data for simulation
rows_of_data = 10000

# Running for multiple network configurations
for i in tqdm(range(rows_of_data)):
    # Initialize the setup
    BS, UEn, UEf, link_bs_uen, link_bs_uef, gain_f, gain_n = initialize()

    # Initialize Links
    # link_bs_uen, link_bs_uef = initialize_links(BS, UEn, UEf)

    # Compute SE for each transmit power for each channel realization for each pair of power_coefficients
    results = computeRates(BS, UEn, UEf, allocation_factors, gain_n, gain_f)

    # Find optimal results
    optimal_run_results = find_optimal_results(results)

    # Prepare data for this run
    run_data = {
        "sample index": i,
        "channel_arrays": [
            link_bs_uen.magnitude.tolist(),  # Channel matrix values for BS-UEn
            link_bs_uef.magnitude.tolist()   # Channel matrix values for BS-UEf
        ],
        "transmit_power (Watt)": BS.t_power.max(),  # Total transmit power of the BS
        "pa_factors_against_that_transmit_power": [
            res['alloc_UEn'] * BS.t_power.max() for res in optimal_run_results  # Power allocated to UEn based on max transmit power
        ],
        "optimal_results": [
            {
                "alloc_UEn": res['alloc_UEn'],
                "alloc_UEf": res['alloc_UEf'],
                "operational_power": res['operational_power'],
                "max_spectral_efficiency": res['max_spectral_efficiency'],
                "SE_difference": res['SE_difference'],
                "optimal_gain_f": res['optimal_gain_f'],
                "optimal_gain_n": res['optimal_gain_n']
            }
            for res in optimal_run_results  # Loop through optimal results
        ]
    }

    # Append this run's data to the list of all runs' data
    data.append(run_data)

# Write all runs' data to a single JSON file at the end of all simulations
write_json_file(data, 'SE_data_10K.json')



  2%|▏         | 216/10000 [00:36<25:01,  6.52it/s]

In [ ]:
# Download json file
from google.colab import files
files.download('SE_data_10K.json')

#Archive


In [13]:
def computeRates(BS, UEn, UEf, allocation_factors, link_bs_uen, link_bs_uef):
  results = []
  sample_index = 0

  # Looping over each allocation factor
  for alloc_UEn in allocation_factors:
      alloc_UEf = 1 - alloc_UEn

      # Update the power allocations for BS
      BS.allocations = {"UEn": alloc_UEn, "UEf": alloc_UEf}

      UEn.sinr_pre = np.zeros((len(Pt), mc))
      UEn.sinr = np.zeros((len(Pt), mc))
      UEf.sinr = np.zeros((len(Pt), mc))
      SE_nf = np.zeros((len(Pt), mc))
      SE_n = np.zeros((len(Pt), mc))
      SE_f = np.zeros((len(Pt), mc))
      SE_total = np.zeros((len(Pt), mc))
      R_n = np.zeros((len(Pt), mc))
      R_f = np.zeros((len(Pt), mc))


      # Get channel gains
      gain_f = link_bs_uef.magnitude**2
      gain_n = link_bs_uen.magnitude**2


      # Loop over each power level
      for i, p in enumerate(Pt_lin):
          p = BS.t_power[i]

          # Loop over each channel realization
          for k in range(mc):
              gain_f_scalar = gain_f[0, 0, k]
              gain_n_scalar = gain_n[0, 0, k]

              # Effective transmit powers for each user
              P_n = BS.allocations["UEn"] * p   # Effective power for near user
              P_f = BS.allocations["UEf"] * p   # Effective power for far user

              # SINR calculations
              SINR_n = (P_n * gain_n_scalar) / N0_lin  # Near user SINR (after SIC)
              SINR_f = (P_f * gain_f_scalar) / (P_n * gain_f_scalar + N0_lin)  # Far user SINR

              # Data rates (using Shannon capacity formula)
              R_n[i, k] = np.log2(1 + SINR_n)  # Data rate for near user
              R_f[i, k] = np.log2(1 + SINR_f)  # Data rate for far user

              # QoS check
              if R_n[i, k] < R_prime_n or R_f[i, k] < R_prime_f:
                  continue  # Skip this allocation if QoS is not met

              sample_index = sample_index + 1

              # Spectral efficiencies
              SE_n[i, k] = np.log2(1 + SINR_n)  # Near user SE
              SE_f[i, k] = np.log2(1 + SINR_f)  # Far user SE

              # Total spectral efficiency
              SE_total[i, k] = SE_n[i, k] + SE_f[i, k]


      # se_index = np.unravel_index(np.argmax(SE_total), SE_total.shape)

      # Finding the optimal transmit power and channel realization for this power-allocation ensuring QOS
      max_se_index = np.unravel_index(np.argmax(SE_total), SE_total.shape)
      optimal_power_index = max_se_index[0]
      optimal_channel_index = max_se_index[1]

      optimal_power = Pt[optimal_power_index]
      max_spectral_efficiency = SE_total[max_se_index]

      # Extract channel realization values for the optimal channel
      optimal_gain_f = gain_f[0, 0, optimal_channel_index]
      optimal_gain_n = gain_n[0, 0, optimal_channel_index]

      # Store results including allocation coefficients and max spectral efficiency
      results.append({
          'sample index' : sample_index,
          'alloc_UEn': alloc_UEn,
          'alloc_UEf': alloc_UEf,
          'operational_power': int(optimal_power),
          "R_n": R_n[i, k],
          "R_f": R_f[i, k],
          'max_spectral_efficiency': max_spectral_efficiency,
          'optimal_gain_f': optimal_gain_f,
          'optimal_gain_n': optimal_gain_n,
          'optimal_channel_index': optimal_channel_index,
          'optimal_power_index': optimal_power_index,
      })

  return results


In [14]:
# # Function to check feasibility of generated locations for UEs
# def is_feasible(bs_position, ue1_position, ue2_position, size_area, max_dist_bs_ue, max_dist_ue_ue, min_dist_ue_ue):
#     # Check if BS and UEs are within the area bounds
#     if (abs(bs_position[0]) > size_area / 2 or abs(bs_position[1]) > size_area / 2 or
#         abs(ue1_position[0]) > size_area / 2 or abs(ue1_position[1]) > size_area / 2 or
#         abs(ue2_position[0]) > size_area / 2 or abs(ue2_position[1]) > size_area / 2):
#         return False

#     # Check distances from BS to UEs
#     if (get_distance(bs_position, ue1_position) > max_dist_bs_ue or
#         get_distance(bs_position, ue2_position) > max_dist_bs_ue):
#         return False

#     # Check distance between UEs
#     if get_distance(ue1_position, ue2_position) > max_dist_ue_ue or get_distance(ue1_position, ue2_position) < min_dist_ue_ue:
#         return False

#     # if UEn is NOT closer to base station than UEf
#     if get_distance(ue1_position, bs_position) < get_distance(ue2_position, bs_position):
#         #print(False)
#         return False

#     #print(get_distance(bs_position, ue1_position) , get_distance(bs_position, ue2_position))

#     return True

# # Initialize Setup
# def initialize():
#   # Initialize Base Station position
#   BS = BaseStation("BS", position=[0, 0, 10], n_antennas=n_antennas, t_power=Pt_lin)

#   # Loop until valid positions for UEn and UEf are found
#   valid_positions_found = False
#   while not valid_positions_found:
#       # Generate random positions for UEn and UEf within specified constraints
#       UEn_position = [np.random.uniform(-max_distance_bs_ue, max_distance_bs_ue),
#                       np.random.uniform(-max_distance_bs_ue, max_distance_bs_ue),
#                       1]

#       UEf_position = [np.random.uniform(-max_distance_bs_ue, max_distance_bs_ue),
#                       np.random.uniform(-max_distance_bs_ue, max_distance_bs_ue),
#                       1]

#       # ADDITIONAL - at same distances
#       # UEf_position = UEn_position

#       # Check if all positions are feasible
#       valid_positions_found = is_feasible(BS.position, UEn_position, UEf_position,
#                                           simulation_area_size, max_distance_bs_ue, max_distance_ue_ue, min_distance_ue_ue)

#       # Ensure UEn is closer to BS than UEf if feasible positions are found
#       if valid_positions_found:
#           dist_UEn_to_BS = getDistance(BS.position[:2], UEn_position[:2])
#           dist_UEf_to_BS = getDistance(BS.position[:2], UEf_position[:2])
#           valid_positions_found = dist_UEn_to_BS <= dist_UEf_to_BS

#   # Initialize User Equipments with their feasible positions
#   UEn = UserEquipment("UEn", position=UEn_position, n_antennas=n_antennas)
#   UEf = UserEquipment("UEf", position=UEf_position, n_antennas=n_antennas)

#   return BS, UEn, UEf


# def getDistance(pos1: list[float], pos2: list[float]) -> float:
#     """Calculate Euclidean distance between two points."""
#     return ((pos1[0] - pos2[0])**2 + (pos1[1] - pos2[1])**2)**0.5

In [15]:
# import numpy as np

# def computeRates(BS, UEn, UEf, allocation_factors, gain_n, gain_f):
#     results = []
#     sample_index = 0

#     # Looping over each allocation factor
#     for alloc_UEn in allocation_factors:
#         alloc_UEf = 1 - alloc_UEn

#         # Update the power allocations for BS
#         BS.allocations = {"UEn": alloc_UEn, "UEf": alloc_UEf}

#         UEn.sinr_pre = np.zeros((len(Pt), mc))
#         UEn.sinr = np.zeros((len(Pt), mc))
#         UEf.sinr = np.zeros((len(Pt), mc))
#         SE_nf = np.zeros((len(Pt), mc))
#         SE_n = np.zeros((len(Pt), mc))
#         SE_f = np.zeros((len(Pt), mc))
#         SE_total = np.zeros((len(Pt), mc))
#         SE_difference = np.zeros((len(Pt), mc))
#         R_n = np.zeros((len(Pt), mc))
#         R_f = np.zeros((len(Pt), mc))


#         # Loop over each power level
#         for i, p in enumerate(Pt_lin):
#             p = BS.t_power[i]

#             # Loop over each channel realization
#             for k in range(mc):
#                 sample_index = sample_index + 1

#                 gain_f_scalar = gain_f[0, 0, k]
#                 gain_n_scalar = gain_n[0, 0, k]

#                 # Effective transmit powers for each user
#                 P_n = BS.allocations["UEn"] * p   # Effective power for near user
#                 P_f = BS.allocations["UEf"] * p   # Effective power for far user

#                 # SINR calculations
#                 SINR_n = (P_n * gain_n_scalar) / N0_lin  # Near user SINR (after SIC)
#                 SINR_f = (P_f * gain_f_scalar) / (P_n * gain_f_scalar + N0_lin)  # Far user SINR

#                 # Data rates (using Shannon capacity formula)
#                 R_n[i, k] = np.log2(1 + SINR_n)  # Data rate for near user
#                 R_f[i, k] = np.log2(1 + SINR_f)  # Data rate for far user


#                 # QoS check
#                 if R_n[i, k] < R_prime_n or R_f[i, k] < R_prime_f:
#                     SE_n[i, k] = 1000  # Near user SE
#                     SE_f[i, k] = 0  # Far user SE

#                 else:
#                   # Spectral efficiencies
#                   SE_n[i, k] = np.log2(1 + SINR_n)  # Near user SE
#                   SE_f[i, k] = np.log2(1 + SINR_f)  # Far user SE


#                 # Total spectral efficiency
#                 SE_total[i, k] = SE_n[i, k] + SE_f[i, k]

#                 # Difference in SE
#                 SE_difference[i, k] = abs(SE_n[i, k] - SE_f[i, k])
#                 # print(SE_difference[i,k])


#         # Find the index where the SE difference is minimized
#         min_diff_index = np.unravel_index(np.argmin(SE_difference), SE_difference.shape)

#         min_diff_power_index = min_diff_index[0]
#         min_diff_channel_index = min_diff_index[1]

#         # Extract the minimum SE difference and corresponding values
#         min_SE_diff = SE_difference[min_diff_index]
#         optimal_power = Pt[min_diff_power_index]
#         optimal_SE_n = SE_n[min_diff_power_index, min_diff_channel_index]
#         optimal_SE_f = SE_f[min_diff_power_index, min_diff_channel_index]
#         optimal_SE_total = SE_total[min_diff_power_index, min_diff_channel_index]
#         optimal_gain_f = gain_f[0, 0, min_diff_channel_index]
#         optimal_gain_n = gain_n[0, 0, min_diff_channel_index]

#         # print(SE_n[min_diff_power_index, min_diff_channel_index], SE_f[min_diff_power_index, min_diff_channel_index], optimal_SE_total)
#         # print(optimal_SE_n, optimal_SE_f)

#         # Store results including allocation coefficients and minimized SE difference
#         results.append({
#             'sample index': sample_index,
#             'alloc_UEn': alloc_UEn,
#             'alloc_UEf': alloc_UEf,
#             'operational_power': int(optimal_power),
#             "SE_n": optimal_SE_n,
#             "SE_f": optimal_SE_f,
#             'SE_difference': min_SE_diff,
#             'max_spectral_efficiency': optimal_SE_total,
#             'optimal_gain_f': optimal_gain_f,
#             'optimal_gain_n': optimal_gain_n,
#             'optimal_channel_index': min_diff_channel_index,
#             'optimal_power_index': min_diff_power_index,
#         })

#     return results